<a href="https://colab.research.google.com/github/frozenscar/NeuralNetFromScratch/blob/master/NeuralNetworkScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [162]:
class NeuralNetwork():
    def __init__(self,dims):
      self.dims = dims
      self.activations = []
      self.init_weights()
      self.dW = []

    def init_weights(self):
      self.W = []
      for l in range(len(self.dims)-1):
          self.W.append(torch.randn(self.dims[l]+1,self.dims[l+1]))

    def printWeights(self):
      print("Weights:")
      for i in range(len(self.W)):
        print(self.W[i])

    def printWeightsShapes(self):
      print("Weights shapes:")
      for i in range(len(self.W)):
        print("Weights ",i,self.W[i].shape)

    def printActivations(self):
      print("Activations:")
      for i in range(len(self.activations)):
        print("Activation ",i,self.activations[i])


    def printActivationsShapes(self):
      print("Activations:")
      for i in range(len(self.activations)):
        print("Activation ",i,self.activations[i].shape)


    def printdWShapes(self):
      print("dW:")
      for i in range(len(self.dW)):
        print("dW ",i,self.dW[i].shape)

    def add_bias(self,X):
      return torch.cat((X,torch.ones(X.shape[0],1)),1)

    def feedForward(self, X):
        self.activations.clear()
        for i in range(len(self.W)):
            X = self.add_bias(X)
            self.activations.append(X)

            X = torch.mm( X, self.W[i])

        return X

    def lossFn(self,y,y_pred):
      loss_gradient = 2*(y_pred-y)

      return loss_gradient


    def backProp(self,X,y):
      self.dW.clear()
      y_pred = self.feedForward(X)

      dLdy = self.lossFn(y,y_pred)
      dLdA = dLdy
      for i in range(len(self.W)):
        #IN the last layer We do not have the bias node
        if i==0:
          self.dW.insert(0,torch.mm(self.activations[-1-i].T,dLdA))

        #We have bias nodes for all the remaining layers,
        #Bias node is not associated with previous layers' weights
        #Hence we remove the gradients associated with bias node.
        else:
          self.dW.insert(0,torch.mm(self.activations[-1-i].T,dLdA)[:,:-1])


        if i==0:
          dLdA = torch.mm(dLdA,self.W[len(self.W)-1-i].T)

        #We will not use the error associated with bias node.
        #because there are no other weights associated going backwards.
        else:
          dLdA = torch.mm(dLdA[:,:-1],self.W[len(self.W)-1-i].T)
      return self.dW

    def updateWeights(self,lr):
      for i in range(len(self.W)):
        self.W[i] = self.W[i] - lr*self.dW[i]

    def train(self,X,y,lr,epochs):
      for i in range(epochs):
        self.backProp(X,y)
        self.updateWeights(lr)
    def predict(self,X):
      return self.feedForward(X)
















In [163]:
nn = NeuralNetwork([2,5,1])
#nn.printWeightsShapes()
X=torch.tensor([[0,0],[0,1],[1,0],[1,1]])
y=torch.tensor([[1],[0],[1],[0]])
nn.feedForward(X)
nn.printActivations()
nn.printActivationsShapes()

nn.backProp(X,y)
nn.printWeightsShapes()
nn.printdWShapes()
nn.train(X,y,0.01,1000)
y = nn.predict(X)

print(y)


Activations:
Activation  0 tensor([[0., 0., 1.],
        [0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 1.]])
Activation  1 tensor([[ 0.5327,  0.4884,  0.6641,  0.7048,  0.6526,  1.0000],
        [-0.0748, -1.1302, -0.6394,  0.0470,  0.7450,  1.0000],
        [ 1.9057,  0.1768,  0.6889,  0.4513,  0.7993,  1.0000],
        [ 1.2982, -1.4417, -0.6145, -0.2066,  0.8916,  1.0000]])
Activations:
Activation  0 torch.Size([4, 3])
Activation  1 torch.Size([4, 6])
Weights shapes:
Weights  0 torch.Size([3, 5])
Weights  1 torch.Size([6, 1])
dW:
dW  0 torch.Size([3, 5])
dW  1 torch.Size([6, 1])
tensor([[ 1.0000e+00],
        [-4.1723e-07],
        [ 1.0000e+00],
        [ 4.3213e-07]])
